In [ ]:
import os, sys
import time
import matplotlib
import matplotlib.pyplot as plt

parentPath = os.path.abspath("../..")
if parentPath not in sys.path:
    sys.path.insert(0, parentPath)

import model as modellib
import visualize
from model import log

%matplotlib inline 

# Root directory of the project
ROOT_DIR = parentPath

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to COCO trained weights
MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

COCO_DIR = "/external_datasets/COCO"

import coco
from dataset import *

def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# coco dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "validation")
dataset.prepare()

## Detection

In [ ]:
class InferenceConfigCoco(coco.CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

# validate that loading conv1 weights from rgb model to rgb-d model works
    
config_rgb = InferenceConfigCoco()
model_rgb = modellib.MaskRCNN(mode="inference", config=config_rgb, model_dir=MODEL_DIR)
model_rgb.load_weights(MODEL_PATH, by_name=True)

conv1 = model_rgb.keras_model.get_layer("conv1")
kernel_rgb, bias = conv1.get_weights()

config_rgbd = InferenceConfigCoco()
config_rgbd.MODE = "RGBD"
config_rgbd.MEAN_PIXEL = np.array([123.7, 116.8, 103.9, 255 / 2])
config_rgbd.IMAGE_SHAPE = np.array([config_rgb.IMAGE_MAX_DIM, config_rgb.IMAGE_MAX_DIM, 4])
model_rgbd = modellib.MaskRCNN(mode="inference", config=config_rgbd,
                          model_dir=MODEL_DIR)
model_rgbd.load_weights(MODEL_PATH, by_name=True, exclude=["conv1"])

conv1 = model_rgbd.keras_model.get_layer("conv1")
kernel_rgbd = np.concatenate((kernel_rgb, np.zeros((7, 7, 1, 64))), axis=2)
conv1.set_weights([kernel_rgbd, bias])

In [ ]:
# Test on a random image
image_id = np.random.choice(dataset.image_ids)
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, config,
                               image_id, use_mini_mask=False)
rgbd = np.dstack((image, np.ones(image.shape[:2]) * 112.5))
visualize.display_instances(image, gt_bbox, gt_mask, gt_class_id,
                            dataset.class_names, figsize=(8, 8))
r = model_rgbd.detect([rgbd])[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset.class_names, r['scores'], ax=get_ax())
r = model_rgb.detect([image])[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset.class_names, r['scores'], ax=get_ax())

## Evaluation

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 10)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox[:,:4], gt_bbox[:,4],
                         r["rois"], r["class_ids"], r["scores"])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))